# Story-writing with BLOOM

SPDX-License-Identifier: 0BSD

In [1]:
from complete import Completer

In [2]:
bloom1 = Completer("""
Sometimes I marvel at how the sky contains no advertisements. Oh, sometimes
someone goes up in a little plane and skywrites a birthday greeting. But the
sky... you don’t have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say, “Thank you for
""", temperature=1.0, top_k=30)

In [3]:
bloom1()

Sometimes I marvel at how the sky contains no advertisements. Oh,
sometimes someone goes up in a little plane and skywrites a birthday
greeting. But the sky... you don’t have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say,
“Thank you for doing your part.” Well yes – but this is a shared
struggle, with history stretching back to the Wright Brothers, to
people like Paul T. Varley. There won’t ever be a time when we are
free of skywriters. It’s a never-ending battle.

The problem began for me one morning, back in the 80s, when the
airways were still considered a commons, before the Reagan-era
deregulation. A local ad man had the bright idea to use a plane to
write an ad for his client on the sky. Then he did the same thing over
at my friend John’s place.

At that time, I was trying to find land to make art on, using a small
plane, and the first thing I noticed about John’s place is the
beautiful view of the valley. He lives on a ridge that rises

In [4]:
bloom2 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

“Aha!” said the biggest, scariest henchman. “This sauce is red! We license the
color red to you, but you haven’t been paying the license fees.” I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""")

In [5]:
bloom2()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

“Aha!” said the biggest, scariest henchman. “This sauce is red! We
license the color red to you, but you haven’t been paying the license
fees.” I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. They were like the police, except they
didn’t care about your driving record. They were the government’s in-
house lawyers, who ripped off your cheese for whatever color they felt
like.


In [6]:
bloom3 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician,
""")

In [7]:
bloom3()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, she knew the power of dreams and the ability of
the subconscious mind to heal itself.

She had experienced this on countless occasions. The problem was that
after a while, the dreams got less and less real. They became more
like fantasy and less like the real thing. Then the energy ghost would
get bored and find a new host.

Beverly’s mother had lived on the planet of Osiris for over 20 years,
and she had just lost her job as a mental health therapist with the
Office of Human Development. The planet was hit hard by the economic
downturn, and Beverly was the only one who still had a job. She was
also the only one who could afford her mother’s help.

Beverly didn’t want to add to the burden of her mother’s debt. She was
a single mother and was struggling to ma

In [8]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom4 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""")

In [9]:
bloom4()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do. Throw some of those sticks down!”

By the time we reached the top of the pass, our group was scattered.
Some of us were climbing, some of us were throwing sticks and others
were in the woods. The survivalist said, “This is the most important
lesson you’re going to learn. The first rule of survival: never stop
moving.”

We all laughed, but he was serious. The survivalist continued, “In my
previous life as an Army Ranger, we lost a man in a jungle training
exercise. We searched for him for days and days, but we never found
him. When they finally extracted us, they found his body. The autopsy
confirmed that he died of exposure. He had just stopped moving and
froze to death.”

The survivalist said, “The first rule of survival: never stop moving.
The second rule of survival: never get off the trail.”

We were having trouble finding footing on t